# General Neural Network

In [1]:
source = "GNN";

In [2]:
using Flux
import BSON

In [3]:
using NBInclude
@nbinclude("Alpha.ipynb");

In [4]:
device = gpu;

## Data preparation

In [5]:
function get_data()
    # inputs are the user's ratings for all shows (unseen shows get mapped to zero) + implicit ratings + heterogenous features
    # outputs are the user's ratings for all shows (unseen shows get mapped to zero)
    df = cat(get_residuals("recommendee", residual_alphas), get_implicit_list())
    X = zeros(Float32, num_items() + 1)
    X[df.item] = df.rating
    Xr = zeros(Float32, num_items() + 1)
    Xr[df.item] .= 1
    X = vcat(X, Xr)

    # add heterogeneous features
    weight = length(df.item)
    nitems_feature = weight / (num_items() + 1)
    push!(X, nitems_feature)
    push!(X, sqrt(nitems_feature))
    push!(X, nitems_feature^2)
    X
end;

In [9]:
# custom split layer
struct Split{T}
    paths::T
end
Split(paths...) = Split(paths)
Flux.@functor Split
(m::Split)(x::AbstractArray) = map(f -> f(x), m.paths)

In [10]:
function compute_alpha()
    BSON.@load "../../data/alphas/$source/model.bson" m
    ratings, implicit = m(get_data())
    write_recommendee_alpha(x -> ratings[x])
    write_recommendee_alpha(x -> implicit[x], outdir = "GNN_implicit")
end;

In [11]:
compute_alpha()